In [1]:
import ipywidgets as widgets
from IPython.display import display
import warnings
import PyPDF2
import datetime
import functools

warnings.filterwarnings('ignore')

# Create the file upload widget
search_file_upload = widgets.FileUpload(
    accept='.pdf',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutputSEARCH = widgets.Output()

txt = widgets.HTML("Upload Files for Searching")

# Display the widget
display(txt, search_file_upload, fileListOutputSEARCH)

ongFileUploadValsSEARCH = []

def on_value_changeSEARCH(change):
    global ongFileUploadValsSEARCH
    global search_file_upload
    
    if (ongFileUploadValsSEARCH ==  [] and list(search_file_upload.value) == []) or ongFileUploadValsSEARCH != list(search_file_upload.value):

        @fileListOutputSEARCH.capture()
        def file_list_output_render():
            global ongFileUploadValsSEARCH
            
            fileListOutputSEARCH.clear_output()
            display(widgets.Label("Uploaded Files:"))
            
            if change != {}:

                # set search_file_upload to previous search_file_upload and current search_file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadValsSEARCH:
                    updatedFileUploadVal.append(file)

                for file in search_file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadValsSEARCH = updatedFileUploadVal
            
                for file in ongFileUploadValsSEARCH:
                    # extract number of pages of uploaded files
                    
                    fileOpen = open(file['name'], 'rb')
                    reader = PyPDF2.PdfReader(fileOpen)
                    totalPgs = len(reader.pages)
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadValsSEARCH
                        # update search_file_upload.value to remove chosen file
                        newVal = []
                        for file in search_file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadValsSEARCH = []
                        search_file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name'] + "  #pages: " + str(totalPgs)), remButton)

                    search_file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

search_file_upload.observe(on_value_changeSEARCH, names='value')

HTML(value='Upload Files for Searching')

FileUpload(value=(), accept='.pdf', description='Upload', multiple=True)

Output()

In [2]:
# Create the file upload widget
examples_file_upload = widgets.FileUpload(
    accept='.txt',  # Accepted file types (e.g., '.txt', '.pdf', '.jpg', '.csv', etc.); leave empty for all file types
    multiple=True  # Set to True if you want to allow multiple files to be uploaded
)

fileListOutput = widgets.Output()

txtSample = widgets.HTML("Upload Files for Sample Data")

# Display the widget
display(txtSample, examples_file_upload, fileListOutput)

ongFileUploadVals = []

def on_value_change(change):
    global ongFileUploadVals
    global examples_file_upload
    
    if (ongFileUploadVals ==  [] and list(examples_file_upload.value) == []) or ongFileUploadVals != list(examples_file_upload.value):

        @fileListOutput.capture()
        def file_list_output_render():
            global ongFileUploadVals
            
            fileListOutput.clear_output()
            display(widgets.Label("Uploaded Files"))
            
            if change != {}:

                # set examples_file_upload to previous examples_file_upload and current examples_file_upload files
                updatedFileUploadVal = []
                for file in ongFileUploadVals:
                    updatedFileUploadVal.append(file)

                for file in examples_file_upload.value:
                    if file not in updatedFileUploadVal:
                        updatedFileUploadVal.append(file)

                # update ong_file_upload value
                ongFileUploadVals = updatedFileUploadVal
            
                for file in ongFileUploadVals:
                    remButton = widgets.Button(description='Remove')
                    
                    def remove_button_onclick(button, fileInfo=""):
                        global ongFileUploadVals
                        # update examples_file_upload.value to remove chosen file
                        newVal = []
                        for file in examples_file_upload.value:
                            if file['name'] != fileInfo:
                                newVal.append(file)
                        ongFileUploadVals = []
                        examples_file_upload.value = tuple(newVal)
                        
                    remButton.on_click(functools.partial(remove_button_onclick, fileInfo=file['name']))
                    
                    display(widgets.Label("\n  file name: " + file['name']), remButton)

                    examples_file_upload.value = tuple(updatedFileUploadVal)

        file_list_output_render()

examples_file_upload.observe(on_value_change, names='value')

HTML(value='Upload Files for Sample Data')

FileUpload(value=(), accept='.txt', description='Upload', multiple=True)

Output()

In [57]:
import re
import fitz
import os
import webbrowser
import spacy
import string
import spacy_transformers

# PART ONE: convert sample data to regex expressions

# https://www.linkedin.com/pulse/programatically-generating-regular-expressions-from-sample-lee-de-byl/
def expr_to_regex(txt, type):
    if type == "0":
        regEx = re.escape(txt)
        regEx = re.sub(r'([^A-Za-z0-9]+)', r'(\1)', regEx)
        regEx = re.sub(r'[A-Za-z]+', lambda matchobj: "([A-Za-z]{"+ str(matchobj.span()[1]-matchobj.span()[0]) +"})", regEx)
        regEx = re.sub(r'[0-9]+', lambda matchobj: "([0-9]{"+ str(matchobj.span()[1]-matchobj.span()[0]) +"})", regEx)
    
        return '^' + regEx + '$'
        
    else:
        regEx = re.escape(txt)
        regEx = re.sub(r'([^A-Za-z0-9]+)', r'(\1)', regEx)
        regEx = re.sub(r'[A-Za-z]+', r'([A-Za-z]+)', regEx)
        regEx = re.sub(r'[0-9]+', r'([0-9]+)', regEx)

        # print("text", txt, "regex", regEx)
        return '^' + regEx + '$'
    
# PART TWO: skim files and redact text that matches regex expressions

# nlp = spacy.load("en_core_web_trf") # slow but accurate
nlp = spacy.load("en_core_web_sm")

def remove_punct(txt):
    return txt.translate(str.maketrans('', '', string.punctuation))
    
def redact_loc_in_file(searchFileUpVal):
    
    # find type of redaction user wants based on first line of file
    redactInfo = dict()
    
    for file in examples_file_upload.value:
         
        # read sample data file
        fileOpen = open(file['name'], "r")
        data = fileOpen.read().split("\n")

        if "SIMPLE REGEX" in data[0].strip():
            # find regex from sample data files
            dataRegEx = []
            for line in data[1:]:
                if expr_to_regex(line, data[0].strip()[-1]) not in dataRegEx:
                    dataRegEx.append(expr_to_regex(line, data[0].strip()[-1]))
            redactInfo[file['name']] = ("regex", dataRegEx)
            
        elif data[0].strip() == "ENTITY TYPE":
            # find entity type from sample data files
            entityLst = []
            for line in data[1:]:
                docLine = nlp(line)
                if docLine.ents:
                    entityLst.append(docLine.ents[0].label_)
            if entityLst:
                redactInfo[file['name']] = ("entity", max(entityLst,key=entityLst.count))
            else:
                redactInfo[file['name']] = ("entity", "O")
        else:
            redactInfo[file['name']] = ("exact", data)
            
    # generate dictionary containing information to be redacted
    redactDict = dict()
    
    # for each example data look at each search file
    for file in searchFileUpVal:
        pdfDoc = fitz.open(file['name'])
        
        for pgNum in range(len(pdfDoc)):
            page = pdfDoc[pgNum]
            
            for exFile in redactInfo:
                matches = []
                
                if exFile not in redactDict:
                    redactDict[exFile] = dict()
                if file['name'] not in redactDict[exFile]:
                    redactDict[exFile][file['name']] = []

                # regex version looks word-by-word
                if redactInfo[exFile][0] == "regex":
                    words = page.get_text("words")
                    
                    # looking words that match regex pattern
                    for regex in redactInfo[exFile][1]:
                        # https://github.com/pymupdf/PyMuPDF/issues/1376
                        search = re.compile(regex) 

                        # 
                        #     for w in words:
                        #         if search.search(w[4]):
                        #             print(w[4], regex, search.search(w[4]))

                        matches += [w[0:4] for w in words if search.search(w[4]) and w[0:4] not in matches]

                # entity version looks phrase-by-phrase
                elif redactInfo[exFile][0] == "entity" and redactInfo[exFile][1] != "O":
                    
                    for block in page.get_text("dict")["blocks"]:
                        if block['type'] == 0:
                            for line in block['lines']:
                                for span in line['spans']:
                                    
                                    text = span['text']
                                    bbox = span['bbox']
                                    words = page.get_text("words", clip=span['bbox'])
                                    # looking for words with same entity
                                    ents = nlp(text).ents 
                                    if ents:
                                        for ent in ents:
                                            if ent.label_ == redactInfo[exFile][1]:
                                                matchesP = list(filter(lambda w: remove_punct(w[4]) in remove_punct(ent.text), words))
                                                # print("matches", matches)
                                                matches += list(map(lambda w: w[0:4], matchesP))
                                        
                # exact version looks word-by-word
                else:
                    # looking for words with exact matches 
                    words = page.get_text("words")

                    matches = [w[0:4] for w in words if w[4] in redactInfo[exFile][1]]
                   
                redactDict[exFile][file['name']] += [(pgNum, match) for match in matches]
    
    return redactDict
        
# display files with redactions

def open_file(filename, pgNum):
    currDir = os.getcwdb()

    # opens file in default browser at specific page, NOTE: assumes that file is in current directory and Chrome is installed
    fileLoc = "file:///" + str(currDir)[2:][:-1] + "/" + filename.replace(" ", "%20") + "#page=" + str(pgNum)

    (webbrowser.get(using='chrome')).open(fileLoc)  
    
# to clear files after you no longer want redactions
fileToClear = ''
def clear_file():
    if fileToClear != '':
        pdfDoc = fitz.open(fileToClear)
    
        # iterates pages and removes annotations generated by fitz
        for pgNum in range(len(pdfDoc)):
            page = pdfDoc[pgNum]
    
            pdfDoc.xref_set_key(page.xref, "Annots", "[]")
        
        pdfDoc.save(fileToClear, incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
        
def clear_btn_click(button): clear_file();
clearBtn = widgets.Button(description='Clear')
clearBtn.on_click(clear_btn_click)

out = widgets.Output(layout={'width': '100%', 'height': '100%'})

def generate_btn_on_click(button): show_results()
generateBtn = widgets.Button(description="Generate Redactions")
generateBtn.on_click(generate_btn_on_click)

display(generateBtn, clearBtn, out)

def redact_matches(doc, redactLst):  
    # iterate through redactions in file 
    foundPgNum = -1
    for (pgNum, bbox) in redactLst:
        if foundPgNum == -1:
            foundPgNum = pgNum + 1
        
        pg = doc[pgNum]
        red = pg.add_rect_annot(bbox)
        red.set_colors(stroke=(0,0,0))
        red.update(opacity=0.5, fill_color=(0,0,0))

    return foundPgNum
    
@out.capture()
def show_results():
    redactDict = redact_loc_in_file(search_file_upload.value)
    
    for exFile in redactDict:
        labelEx = widgets.HTML(value="<style>p{word-wrap: break-word}</style> <p> Redaction Expressions File Name: " + exFile + "</p>")
        display(labelEx)

        for searchFile in redactDict[exFile]:
            if redactDict[exFile][searchFile]:
                labelSearch = widgets.HTML(value="<style>p{word-wrap: break-word}</style> <p> File Name: " + searchFile + "</p>")
                display(labelSearch)
    
                redactButton = widgets.Button(description='Redact')

                def on_redact_button_click(button, info=(dict(), "", "")):
                    global fileToClear
                    clear_file()
                    
                    doc = fitz.open(info[2])
                    foundPgNum = redact_matches(doc, info[0][info[1]][info[2]])
                    doc.save(info[2], incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP)
                    
                    fileToClear = info[2]
                    open_file(info[2], foundPgNum)
    
                redactButton.on_click(functools.partial(on_redact_button_click, info=(redactDict, exFile, searchFile)))
                display(redactButton)

Button(description='Generate Redactions', style=ButtonStyle())

Button(description='Clear', style=ButtonStyle())

Output(layout=Layout(height='100%', width='100%'))

In [4]:
# import spacy

# NER = spacy.load("en_core_web_trf")
# raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
# docLine = NER(raw_text)
# for ent in docLine.ents:
#     print(ent.text, ent.label_)

The Indian Space Research Organisation ORG
India GPE
Bengaluru GPE
Department of Space ORG
India GPE
ISRO ORG
DOS ORG


In [31]:
# def id_to_regex(id):
#     x = re.escape(id)
#     # x = re.sub(r"(@)", r'(\1)', x)
#     x = re.sub(r'([^A-Za-z0-9]+)', r'(\1)', x)
#     x = re.sub(r'[A-Za-z]+', r'([A-Za-z]+)', x)
#     x = re.sub(r'[0-9]+', r'([0-9]+)', x)
#     return '^' + x + '$'

# print(id_to_regex("Iman"))

^([A-Za-z]+)$
